In [ ]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import json
import warnings
warnings.filterwarnings(action='once')

'''
Example format for extraction:
Count | Card Name | (Set# Card#)

2 Trail Stories (Set1 #188)
4 Horus Traver (Set1002 #23)
4 Longbarrel (Set4 #5)
4 On the Hunt (Set2 #5)
4 Oni Dragonsmith (Set1003 #2)
'''

# First step: Login and extract collection fully from eternal warcry.

## Note: Remove user/pass lane later!

In [7]:
"""
For those paying attention mechanize is python 2.x only, someone made mechanicalsoup
(Love you), which if you read the tutorial we give you a brief intro along with show you how
to login. This is compatible /w python 3.6. The code below takes in two command
line parameterse delimited by a space; username password.

ie: aggregate_card_json.py <username> <password>

For now I have my u+p information manually punched in and works!
"""

# import argparse
# parser = argparse.ArgumentParser(description="Login to EternalWarcry.")
# parser.add_argument("username")
# parser.add_argument("password")
# args = parser.parse_args()

# Account info for Eternal Warcry site
user = '<user>'
passwd = '<pass>'

import mechanicalsoup

browser = mechanicalsoup.Browser(soup_config={'features': 'lxml'})

# request eternalWarcry login page. the result is a requests.Response object
# login_page.soup is a BeautifulSoup object
login_page = browser.get("https://eternalwarcry.com/login")

# we grab the login form
login_form = mechanicalsoup.Form(login_page.soup.select_one('form[action="/login"]'))

# specify username and password
login_form.input({"LoginEmail": user, "LoginPassword": passwd})# REPLACE WITH args.username & args.password

# submit form
page2 = browser.submit(login_form, login_page.url)

"""
STEP TWO: ITERATE THROUGH EACH PAGE AND GRAB ALL OF THE INFORMATION FOR EACH CARD,
PUT IT INTO A STRING IN THE PRETTY FORMAT THAT ETERNAL FOLLOWS, AND APPEND INTO A BIG LIST
OUTPUT WHICH IS YOUR COLLECTION FROM ETERNALWARCRY
(All this really does is save you time by not having to go into ->"Deck Builder" and
add ALL your cards to a deck and exporting it.... :))
"""
page_num = 1
final_collection = []

while True:
    # Now that were logged in pull each link for our collection and grab the info
    current_page = 'https://eternalwarcry.com/cards?p=' + str(page_num)
    page_num = page_num + 1 # Iterate don't wanna forget ;)
    collection_page = browser.get(current_page) # Response object with .soup object
    
    # Current Page. #collection_page.text because its response object
    html_page = collection_page.soup 
    # Get all cards on this page.
    divs = html_page.findAll(class_= 'card-search-item col-lg-3 col-sm-4 col-xs-6 add-card-main element-relative')  
    
    # Get out of while loop when on last page.
    if not divs:
        # This means were on the last page of cards, (56 at this time) and there are no card objects. Finish.
        break
            
    for div in divs:
        card_details = str(div.find_all('a')) # Main card object
        # Now get card name with grammar
        name = str(re.search(r'<img alt="(.+)" class=', card_details, re.IGNORECASE).group(1))
        end_of_url = str(div.find('a').get('href'))
        full_card_url = 'https://eternalwarcry.com' + end_of_url

        # Put it altogether....
        item = [name, full_card_url]

        # Drop each page into a list for later...
        final_collection.append(item)

Eternal Warcry | Eternal Card Game Decks and Card Browser
BEGIN
END


In [9]:
len(final_collection) # At this time there are 1754 cards. Success!

1754

In [10]:
total_cards = len(final_collection)
output_json = {'total_cards': total_cards, 'all_cards': final_collection} 
# all_cards = A list containing lists, where each has card name and url

In [11]:
# Cool! WE got an accurate record of our cards, minus the Commons and UnCommons. Success for tonight.
# Output to a JSON file because sexy
with open('all_cards.json', 'w') as fp:
    json.dump(output_json, fp)